In [71]:
# # Install runtime dependencies (safe to skip if your env is already prepared)
# %pip install -U pip
# %pip install ninja setuptools wheel

# # PyTorch + CUDA stack used by the SMH adapters
# %pip install "torch==2.4.0" "torchvision==0.19.0" "torchaudio==2.4.0"   --index-url https://download.pytorch.org/whl/cu121

# # FlashAttention + HF stack (mamba_ssm comes from this repo via editable install)
# %pip install "flash-attn==2.6.3" --no-build-isolation
# %pip install "transformers==4.47.0" "accelerate<1.0" "huggingface_hub" einops

# # Make this checkout importable so trust_remote_code loads the custom architecture
# %pip install -e .


In [72]:
import os
from pathlib import Path
from huggingface_hub import snapshot_download

REPO_ROOT = Path(os.environ.get("MAMBA_EVAL_ROOT", Path.cwd())).resolve()
CHECKPOINT_DIR = Path(
    os.environ.get(
        "MAMBA_ADAPTER_CKPT",
        REPO_ROOT / "outputs/reflong_mamba_adapter/checkpoint-500",
    )
).resolve()
HF_REPO_ID = os.environ.get("MAMBA_ADAPTER_HF", "htang08/shm-ssm")
HF_REVISION = os.environ.get("MAMBA_ADAPTER_REVISION", "main")
DOWNLOAD_IF_MISSING = os.environ.get("MAMBA_ADAPTER_DOWNLOAD", "0") == "1"

print(f"Repo root: {REPO_ROOT}")
print(f"Planned checkpoint: {CHECKPOINT_DIR}")

if not CHECKPOINT_DIR.exists():
    if not DOWNLOAD_IF_MISSING:
        raise FileNotFoundError(
            "Checkpoint folder not found. Set MAMBA_ADAPTER_CKPT or enable download by"
            " exporting MAMBA_ADAPTER_DOWNLOAD=1."
        )
    print(f"Checkpoint missing – downloading {HF_REPO_ID}@{HF_REVISION} via huggingface_hub...")
    CHECKPOINT_DIR = Path(
        snapshot_download(
            repo_id=HF_REPO_ID,
            revision=HF_REVISION,
            local_dir=str(REPO_ROOT / "downloaded_adapter"),
            local_dir_use_symlinks=False,
        )
    )

MODEL_DIR = CHECKPOINT_DIR
print(f"Using checkpoint directory: {MODEL_DIR}")


Repo root: /storage/ice1/6/8/htang318/mamba
Planned checkpoint: /storage/ice1/6/8/htang318/mamba/outputs/reflong_mamba_adapter/checkpoint-500
Using checkpoint directory: /storage/ice1/6/8/htang318/mamba/outputs/reflong_mamba_adapter/checkpoint-500


In [73]:
# Optional: uncomment if you need to authenticate to download from a private HF repo.
# from huggingface_hub import login
# login()


In [74]:
!nvidia-smi

Tue Nov 18 13:44:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.57.08              Driver Version: 575.57.08      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          On  |   00000000:40:00.0 Off |                    0 |
| N/A   32C    P0            129W /  700W |   19971MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [75]:
# import os
# from pathlib import Path

# ADA_LEVAL_DIR = (REPO_ROOT / "Ada-LEval").resolve()
# if not ADA_LEVAL_DIR.exists():
#     !git clone https://github.com/open-compass/Ada-LEval.git "{ADA_LEVAL_DIR}"

# %cd "{ADA_LEVAL_DIR}"
# if Path("fetch_data.sh").exists():
#     !bash fetch_data.sh
# else:
#     print("WARNING: fetch_data.sh not found – make sure data/*.json exists.")

# %cd "{REPO_ROOT}"


In [76]:
import os
from pathlib import Path


os.chdir(REPO_ROOT)
print("Working directory:", Path.cwd())


Working directory: /storage/ice1/6/8/htang318/mamba


In [77]:
import json
import re
import time
from pathlib import Path

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.auto import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

# Use TF32 for speed on A100
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# How many examples per task to run (change if needed)
MAX_SAMPLES_PER_TASK = 250

# Generation parameters
MAX_NEW_TOKENS = 32   # enough for "Answer: [4,1,3,2]" etc.
TEMPERATURE = 0.0     # greedy (deterministic)

Device: cuda


In [78]:
import json
import itertools
from collections import Counter
import time
import torch


In [79]:
DATA_DIR = ADA_LEVAL_DIR / "data"

def load_adaleval_task(task_name: str, max_samples: int = None):
    task_path = DATA_DIR / f"{task_name}.json"
    if not task_path.exists():
        raise FileNotFoundError(f"Missing task file: {task_path}")
    with open(task_path, "r") as f:
        data = json.load(f)
    if max_samples:
        data = data[:max_samples]
    return data


In [80]:
import json
import torch
from transformers import AutoTokenizer

from mamba_ssm.models.config_mamba import MambaConfig
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.bfloat16 if DEVICE == "cuda" else torch.float32

# Load tokenizer saved with the checkpoint
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Build config from the saved json using the local dataclass
config_path = MODEL_DIR / "config.json"
with open(config_path, "r") as f:
    raw_cfg = json.load(f)
allowed = set(MambaConfig.__dataclass_fields__.keys())
filtered_cfg = {k: v for k, v in raw_cfg.items() if k in allowed}
config = MambaConfig(**filtered_cfg)

# Instantiate model from local implementation and load weights
model = MambaLMHeadModel(config, dtype=dtype, device=DEVICE)
state_path = MODEL_DIR / "pytorch_model.bin"
state = torch.load(state_path, map_location="cpu")
if dtype != torch.float32:
    for k, v in list(state.items()):
        if torch.is_tensor(v):
            state[k] = v.to(dtype)
missing, unexpected = model.load_state_dict(state, strict=False)
print(f"Missing keys: {len(missing)} | Unexpected keys: {len(unexpected)}")

model.to(device=DEVICE, dtype=dtype)
model.eval()
print(f"Model ready on {DEVICE} with dtype {dtype}.")


/tmp/ipykernel_1687089/3991378625.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(state_path, map_location="cpu")


Missing keys: 0 | Unexpected keys: 0
Model ready on cuda with dtype torch.bfloat16.


In [81]:
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

if "model" not in globals():
    print("Model not loaded yet – run the model-loading cell before this check.")
else:
    base_model = getattr(model, "model", model)
    print("Loaded model type:", type(base_model))
    if isinstance(base_model, MambaLMHeadModel):
        adapter = getattr(base_model.backbone.layers[0], "adapter", None)
        print("Adapter active:", adapter.is_active if adapter is not None else False)
    else:
        print("Warning: model is not a MambaLMHeadModel – ensure the custom weights loaded correctly.")


Loaded model type: <class 'mamba_ssm.models.mixer_seq_simple.MambaLMHeadModel'>
Adapter active: True


In [82]:
import json

config_path = MODEL_DIR / "config.json"
with open(config_path, "r") as f:
    config_json = json.load(f)

print("Adapter config present:", "adapter_cfg" in config_json)
if "adapter_cfg" in config_json:
    print(json.dumps(config_json["adapter_cfg"], indent=2)[:500])


Adapter config present: True
{
  "saliency_gate": {
    "enabled": true,
    "hidden_dims": [
      64,
      256
    ],
    "dropout": 0.1,
    "clamp_range": [
      0.05,
      0.95
    ],
    "train_only": true
  },
  "selective_memory": {
    "enabled": true,
    "window_sizes": [
      2
    ],
    "memory_dim": 128,
    "dropout": 0.1
  }
}


In [83]:
print("Using the local editable install of mamba_ssm from this checkout.")


Using the local editable install of mamba_ssm from this checkout.


In [84]:
import torch
import torch.nn as nn

class HFCompatMamba(nn.Module):
    def __init__(self, model, tok):
        super().__init__()
        self.model = model
        self.tok = tok

    def forward(self, **kwargs):
        # strip unsupported keys
        remove = [
            "attention_mask", "position_ids", "token_type_ids",
            "do_sample", "temperature", "top_k", "top_p",
            "repetition_penalty", "use_cache",
        ]
        for k in remove:
            kwargs.pop(k, None)
        return self.model(**kwargs)

    def generate(self, input_ids=None, max_new_tokens=None, **kwargs):
        # strip HF-only / unused keys Ada-LEval passes
        remove = [
            "attention_mask", "position_ids", "token_type_ids",
            "do_sample", "temperature", "top_k", "top_p",
            "repetition_penalty", "use_cache",
            "return_dict_in_generate", "output_scores",
        ]
        for k in remove:
            kwargs.pop(k, None)

        if max_new_tokens is None:
            max_new_tokens = kwargs.pop("max_new_tokens", 64)

        kwargs.pop("max_length", None)
        max_length = input_ids.shape[1] + max_new_tokens

        return self.model.generate(
            input_ids=input_ids,
            max_length=max_length,
            **kwargs,
        )

# Wrap the HF model so Ada-LEval can call it
model = HFCompatMamba(model, tokenizer)


In [85]:
adapter_cfg = getattr(getattr(model, "model", model).config, "adapter_cfg", None)
if not adapter_cfg:
    raise RuntimeError("Adapter configuration missing – make sure MODEL_DIR points to your finetuned checkpoint.")
print("Adapter configuration detected. Selective memory windows:", adapter_cfg.get("selective_memory", {}).get("window_sizes"))


Adapter configuration detected. Selective memory windows: [2]


In [86]:
import re


def textsort_extract_prediction(prediction: str):
    match = re.search(r"\[([^\]]+)\]", prediction)
    candidate = match.group(1) if match else prediction
    nums = re.findall(r"-?\d+", candidate)
    return [int(n) for n in nums]


def stackselect_extract_prediction(prediction: str, num_choices: int):
    matches = re.findall(r"[Aa](\d+)", prediction)
    for m in matches:
        idx = int(m)
        if 1 <= idx <= num_choices:
            return f"A{idx}"
    return "A1" if num_choices else "A1"


def f1_lists(prediction, gold):
    pred_set = set(prediction)
    gold_set = set(gold)
    if not pred_set and not gold_set:
        return 1.0
    if not pred_set or not gold_set:
        return 0.0
    true_pos = len(pred_set & gold_set)
    precision = true_pos / len(pred_set)
    recall = true_pos / len(gold_set)
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)


In [87]:
@torch.inference_mode()
def eval_model_on_adaleval_task(
    model,
    tok,
    task_name: str,
    max_samples: int = MAX_SAMPLES_PER_TASK,
    max_new_tokens: int = MAX_NEW_TOKENS,
    device: str = DEVICE,
):
    samples = load_adaleval_task(task_name, max_samples=max_samples)
    n = len(samples)
    print(f"\n=== Task: {task_name} | num_samples = {n} ===")

    is_textsort = task_name.startswith("textsort")
    is_stackselect = task_name.startswith("stackselect")

    total_tokens = 0
    num_correct = 0.0
    f1_sum = 0.0  # for TextSort; for StackSelect this will equal accuracy

    if device == "cuda":
        torch.cuda.reset_peak_memory_stats()

    start_time = time.perf_counter()

    for i, sample in enumerate(tqdm(samples, desc=f"{task_name}", unit="ex")):
        # ----- Build prompt (mirror Ada-Leval) -----
        if is_textsort:
            # TextSort stores the full prompt string in the JSON already
            prompt = sample["prompt"]
        elif is_stackselect:
            # Reconstruct prompt like StackSelect.build_prompt
            question = sample["question"]
            all_answers = sample["all_answers"]

            meta_prompt = """
You are an AI assistant. Your job is to find out the most helpful answer to a given question.
Each time, you will be provided with a question and n answers to this question.
Each answer begins with an 'A' and a number(e.g. A4), which represents its designation.
You need to determine which answer is the most helpful one to the question.
The case sample is shown below and you should give me the answer in the format exactly the same as the sample. \n
However, you should NOT focus on the content of sample answer. \n
Sample Input (format only): \n
The question is given below.
XXX(The content of question)
Possible answers are given below.
A1:
XXX(The content of answer 1)
A2:
XXX(The content of answer 2)
.
.
.
An:
XXX(The content of answer n)
Now the answers are over, please decide which answer is the most helpful one to the question.
You must give me only the designation of the MOST helpful answer.
Sample Output (format only): \n
Answer: The designation of the most helpful answer.(e.g. A4 means answer 4 is the most helpful answer) \n\n
"""

            prompt = meta_prompt
            prompt += "The question is given below.\n"
            prompt += question + "\n\n"
            prompt += "Possible answers are given below.\n"
            for j, ans in enumerate(all_answers, start=1):
                prompt += f"A{j}:\n\n{ans}\n\n"
            prompt += """
Now the answers are over, please decide which answer is the most helpful one to the question.
You must give me only the designation of the MOST helpful answer.
"""
        else:
            raise ValueError(f"Unknown Ada-LEval task type: {task_name}")

        # ----- Tokenize & generate -----
        inputs = tok(prompt, return_tensors="pt", truncation=False).to(device)
        input_len = inputs["input_ids"].shape[1]

        out = model.generate(
            **inputs,
            do_sample=False,
            temperature=None,
            use_cache=True,
            max_length=4000
        )

        full_seq = out[0]
        gen_ids = full_seq[input_len:]  # generated tokens only

        total_tokens += gen_ids.numel()

        pred_text = tok.decode(gen_ids, skip_special_tokens=True)

        # ----- Gold + extraction -----
        if is_textsort:
            # Gold answer: list or JSON string
            gold = sample["answer"]
            if isinstance(gold, str):
                gold = json.loads(gold)

            pred_extracted = textsort_extract_prediction(pred_text)

            # EM
            correct = int(list(pred_extracted) == list(gold))
            num_correct += correct

            # F1 (order-insensitive) for extra signal
            f1_sum += f1_lists(pred_extracted, gold)

        elif is_stackselect:
            # Gold is like "A4"
            gold = sample["answer"]
            num_choice = len(sample["all_answers"])

            pred_extracted = stackselect_extract_prediction(pred_text, num_choice)

            correct = int(pred_extracted == gold)
            num_correct += correct
            # For single-label classification, F1 per example is 1 if correct else 0,
            # so average F1 == accuracy.
            f1_sum += correct

        # Optional logging
        if (i + 1) % 10 == 0:
            print(
                f"[{i + 1}/{n}] "
                f"Acc: {num_correct / (i + 1):.4f} | "
                f"F1: {f1_sum / (i + 1):.4f}"
            )

    elapsed = time.perf_counter() - start_time
    elapsed = max(elapsed, 1e-8)

    if device == "cuda":
        peak_mem_gb = torch.cuda.max_memory_allocated() / (1024 ** 3)
    else:
        peak_mem_gb = 0.0

    accuracy = num_correct / n
    mean_f1 = f1_sum / n
    throughput = total_tokens / elapsed

    print(f"Total accuracy: {accuracy:.4f}")
    print(f"Total F1:       {mean_f1:.4f}")
    print(f"Throughput:     {throughput:.2f} tokens/sec")
    print(f"Peak memory:    {peak_mem_gb:.2f} GB")

    return {
        "task": task_name,
        "num_examples": n,
        "accuracy": accuracy,          # matches Ada-Leval logic
        "f1": mean_f1,                 # extra metric (TextSort real F1, StackSelect == acc)
        "throughput_toks_per_sec": throughput,
        "peak_mem_gb": peak_mem_gb,
    }


In [88]:
SAVE_PATH = "./adaleval_results.json"
MODEL_LABEL = MODEL_DIR.name  # will show up in the summary table

TASK_NAMES = [
    # "stackselect_1k",
    # "stackselect_4k",
    # "stackselect_8k",
    "textsort_1k",
    "textsort_2k"
]

if os.path.exists(SAVE_PATH):
    with open(SAVE_PATH, "r") as f:
        all_results = json.load(f)
    print(f"Loaded {len(all_results)} existing results from {SAVE_PATH}")
else:
    all_results = []

def save_results_to_json():
    with open(SAVE_PATH, "w") as f:
        json.dump(all_results, f, indent=2)
    print(f"✔ Saved {len(all_results)} results to {SAVE_PATH}")

for task_name in TASK_NAMES:
    if any(r.get("model") == MODEL_LABEL and r.get("task") == task_name for r in all_results):
        print(f"Skipping {MODEL_LABEL} on {task_name} (already done).")
        continue

    print(f"\n▶ Running {MODEL_LABEL} on task: {task_name}")
    res = eval_model_on_adaleval_task(
        model,
        tokenizer,
        task_name=task_name,
        max_samples=50,
    )

    res["model"] = MODEL_LABEL
    res.setdefault("f1", res.get("accuracy", 0.0))

    all_results.append(res)
    save_results_to_json()

import pandas as pd

df = pd.DataFrame(all_results)
expected_cols = [
    "model", "task", "num_examples", "accuracy", "f1",
    "throughput_toks_per_sec", "peak_mem_gb"
]
for col in expected_cols:
    if col not in df.columns:
        df[col] = float("nan")

df = df[expected_cols]
df.sort_values(["model", "task"], inplace=True)
df.reset_index(drop=True, inplace=True)
df


Loaded 3 existing results from ./adaleval_results.json

▶ Running checkpoint-500 on task: textsort_1k

=== Task: textsort_1k | num_samples = 50 ===


textsort_1k:   0%|          | 0/50 [00:00<?, ?ex/s]

textsort_1k:  20%|██        | 10/50 [00:26<01:46,  2.66s/ex]

[10/50] Acc: 0.0000 | F1: 0.0000


textsort_1k:  40%|████      | 20/50 [00:53<01:20,  2.68s/ex]

...

textsort_2k:  80%|████████  | 40/50 [01:48<00:27,  2.76s/ex]

[40/50] Acc: 0.0000 | F1: 0.0000


textsort_2k: 100%|██████████| 50/50 [02:15<00:00,  2.72s/ex]

[50/50] Acc: 0.0000 | F1: 0.0000
Total accuracy: 0.0000
Total F1:       0.0000
Throughput:     23.57 tokens/sec
Peak memory:    16.61 GB
✔ Saved 5 results to ./adaleval_results.json


,model,task,num_examples,accuracy,f1,throughput_toks_per_sec,peak_mem_gb
0,checkpoint-500,stackselect_1k,50,0.32,0.32,24.388601,16.813341
1,checkpoint-500,stackselect_4k,50,0.04,0.04,22.795091,17.156179
2,checkpoint-500,stackselect_8k,50,0.04,0.04,20.425367,17.531181
3,checkpoint-500,textsort_1k,50,0.00,0.00,24.057186,16.516158
4,checkpoint-500,textsort_2k,50,0.00,0.00,23.566205,16.607120
